In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
from gvopt import *
from math import pi
import matplotlib.pyplot as plt
import matplotlib.colors as mc

# Analytical tests for natural-attenuation ABCs

## Define test parameters

In [ ]:
# define frequency samples being considered
f0 = 5.0
f1 = 50.0
nf = 10
fs = np.linspace(f0,f1,nf)
w = fs*2*pi

In [ ]:
# define modeling parameters
d = 20 # grid cell size
dt = 2e-3 # time interval
w0 = 20*2*pi # reference angular frequency

In [ ]:
# define boundary parameter
vb = 2000 # boundary velocity
gb = np.arctan(1/100)/pi # boundary gamma (corresponding to Q=100)

In [ ]:
# overall tolerance
eps1 = 0.04
eps2 = 0.01

## Define NAABC classes

In [ ]:
test1 = vgopt_model(2, d, dt, w0, w, eps1) # NAABC global class
test2 = vgopt_model(2, d, dt, w0, w, eps2) 
testgb1 = vgopt(vb,gb,mp=test1) # NAABC boundary pair class
testgb2 = vgopt(vb,gb,mp=test2) 

In [ ]:
# display the stability factors in test1
sg = test1.s.sf
g = test1.s.gs

fig, ax = plt.subplots(1,1,figsize=(6,4))
ax.plot(g,sg,linewidth=2,color='r')
ax.set_xlabel('$\gamma$', fontsize=20)
ax.set_ylabel('Stability factor', fontsize=20)

xtick = np.linspace(0,0.5,6)
ytick = np.linspace(0.2,0.6,5)
ax.set_xticks(xtick)
ax.set_xticklabels([f'{i:.1f}' for i in xtick],fontsize=15)
ax.set_yticks(ytick)
ax.set_yticklabels([f'{i:.1f}' for i in ytick],fontsize=15)

for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(2)

In [ ]:
BB = ax.get_position()

In [ ]:
BB = ax.get_position()
BB.x0 -= 0.3 
BB.y0 -= 0.3
BB.x1 = 6.5
BB.y1 = 4.5

In [ ]:
fig.savefig('./outputs/AGUpaper/sg.png',dpi=600,bbox_inches=BB)

## Perform optimization

In [ ]:
N1,va1,ga1,epsi1 = testgb1.Ncal()
N2,va2,ga2,epsi2 = testgb2.Ncal()

## Display results: test 1

In [ ]:
N = N1
va = va1
ga = ga1
epsi = epsi1
ytk = np.arange(1800,2401,200)

# display optimized va and ga
x = np.arange(1,N+1)

fig, ax1 = plt.subplots(1,1,figsize=(8.2,3.55))

color = 'tab:red'
ax1.set_xlabel('Absorbing layer No.', fontsize=20)
ax1.set_ylabel('$v_a$ (m/s)', color=color, fontsize=20)
ax1.plot(x, va, color=color, linewidth=2)
ax1.tick_params(axis='y', labelcolor=color)

xtick = np.array(np.linspace(1,N,5),dtype=np.int16)
ytick = ytk
ax1.set_xticks(xtick)
ax1.set_xticklabels([f'{i}' for i in xtick],fontsize=15)
ax1.set_yticks(ytick)
ax1.set_yticklabels([f'{i}' for i in ytick],fontsize=15)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:blue'
ax2.set_ylabel('$\gamma_a$', color=color, fontsize=20)  # we already handled the x-label with ax1
ax2.plot(x, ga, color=color, linewidth=2)
ax2.tick_params(axis='y', labelcolor=color)

ytick = np.arange(0,0.51,0.1)
ax2.set_yticks(ytick)
ax2.set_yticklabels([f'{i:.1f}' for i in ytick],fontsize=15)

fig.tight_layout()  # otherwise the right y-label is slightly clipped

for axis in ['top','bottom','left','right']:
    ax1.spines[axis].set_linewidth(2)
    
plt.show()

In [ ]:
fig.savefig('./outputs/AGUpaper/vga_eps=0.04.png',dpi=300)

In [ ]:
wi = np.linspace(w[0],w[-1],101)
# calculate the dispersive velocity
vd = np.zeros((N,len(wi)))
for i in range(N):
    vd[i,:] = va[i]*np.cos(0.5*pi*ga[i])*(wi/w0)**ga[i]

In [ ]:
def nl(i,N,cmax=0.6):
    return np.sqrt(i/(N-1)*cmax)
xtk = np.linspace(5,50,6)
ytk = np.arange(800,2801,400)
# display the dispersive velocity
fig, ax = plt.subplots(1,1,figsize=(8,6))
for i in range(N):
    c = nl(i,N)
    if i == 0:
        ax.plot(wi/(2*pi),vd[i,:],color=(c,c,c),label='layer 1')
        continue
    if i == N-1:
        ax.plot(wi/(2*pi),vd[i,:],color=(c,c,c),label=f'layer {i+1}')
        continue
    if (i % 3 == 0) and (i!=15):
        ax.plot(wi/(2*pi),vd[i,:],color=(c,c,c),label=f'layer {i+1}')
    ax.plot(wi/(2*pi),vd[i,:],color=(c,c,c))
for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(2)
ax.legend(fontsize=15)
ax.set_xticks(xtk)
ax.set_xticklabels([f'{int(i)}' for i in xtk],fontsize=15)
ax.set_yticks(ytk)
ax.set_yticklabels([f'{i}' for i in ytk],fontsize=15)
ax.set_xlabel('Frequency (Hz)',fontsize=20)
_ = ax.set_ylabel('Velocity (m/s)',fontsize=20)

In [ ]:
fig.savefig('./outputs/AGUpaper/vdf_eps=0.04.png',dpi=600)

In [ ]:
# display returning residuals
y = testgb1.ABCtest(va, ga, disp=False)
nw = len(w)
Vmin = np.amin(y)
Vmax = np.amax(y)
cN = pcl.Normalize(vmin=Vmin, vmax=Vmax)
fig, ax = plt.subplots(1,1,figsize=(10,3.55))
ax.imshow(y,cmap='gray',origin='lower',\
          extent=[1,N,f0,f1],aspect=3/7*(N-1)*2*np.pi/(w[-1]-w[0]))
ax.set_xlabel('Absorbing layer No.',fontsize=20)
ax.set_ylabel('Frequency (Hz)',fontsize=20)
ax.set_xlim(1,N)
ax.set_ylim(5,50)
xtick = np.array(np.linspace(1,N,5),dtype=np.int16)
ytick = np.linspace(f0,f1,6)
ax.set_xticks(xtick)
ax.set_xticklabels([f'{i}' for i in xtick],fontsize=15)
ax.set_yticks(ytick)
ax.set_yticklabels([f'{i:.1f}' for i in ytick],fontsize=15)

cb = fig.colorbar(cm.ScalarMappable(norm=cN, cmap='gray'))
ctick = np.linspace(Vmin,Vmax,5)
cb.set_ticks(ctick)
cb.set_ticklabels([f'{i*1e2:.2f}' for i in ctick])
cb.ax.tick_params(labelsize=15)
pos = list(cb.ax.get_position().bounds)
pos[0] += 0.05
cb.ax.set_position(pos)
cb.set_label('$A_i(\omega)$',fontsize=15)

ax2 = ax.twinx()
color = 'tab:red'
ax2.set_ylabel('$\epsilon_i$', color=color, fontsize=20)
ax2.plot(x, epsi[1:], color=color, linewidth=2)
ax2.tick_params(axis='y', labelcolor=color)
ax2.set_ylim(0,0.04)
ytick = np.linspace(0,0.04,5)
ax2.set_yticks(ytick)
ax2.set_yticklabels([f'{i*100:.1f}' for i in ytick],fontsize=15)

ax2.text(16.6, 0.042, 'x1e-2',fontsize=15,color=color)
ax2.text(19.6, 0.042, 'x1e-2',fontsize=15)

for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(2)
#fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()

In [ ]:
BB = ax.get_position()
BB.x0 = 0
BB.y0 = -0.3
BB.x1 = 10
BB.y1 = 3.55
fig.savefig('./outputs/AGUpaper/Aepsi_eps=0.04.png',dpi=600,bbox_inches=BB)

## Display results: test 2

In [ ]:
N = N2
va = va2
ga = ga2
epsi = epsi2
ytk = np.arange(1800,2401,200)
# display optimized va and ga
x = np.arange(1,N+1)

fig, ax1 = plt.subplots(1,1,figsize=(8.2,3.55))

color = 'tab:red'
ax1.set_xlabel('Absorbing layer No.', fontsize=20)
ax1.set_ylabel('$v_a$ (m/s)', color=color, fontsize=20)
ax1.plot(x, va, color=color, linewidth=2)
ax1.tick_params(axis='y', labelcolor=color)

xtick = np.array(np.linspace(1,N,5),dtype=np.int16)
ytick = ytk
ax1.set_xticks(xtick)
ax1.set_xticklabels([f'{i}' for i in xtick],fontsize=15)
ax1.set_yticks(ytick)
ax1.set_yticklabels([f'{i}' for i in ytick],fontsize=15)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:blue'
ax2.set_ylabel('$\gamma_a$', color=color, fontsize=20)  # we already handled the x-label with ax1
ax2.plot(x, ga, color=color, linewidth=2)
ax2.tick_params(axis='y', labelcolor=color)

ytick = np.arange(0,0.51,0.1)
ax2.set_yticks(ytick)
ax2.set_yticklabels([f'{i:.1f}' for i in ytick],fontsize=15)

fig.tight_layout()  # otherwise the right y-label is slightly clipped

for axis in ['top','bottom','left','right']:
    ax1.spines[axis].set_linewidth(2)
    
plt.show()

In [ ]:
fig.savefig('./outputs/AGUpaper/vga_eps=0.01.png',dpi=300)

In [ ]:
wi = np.linspace(w[0],w[-1],101)
# calculate the dispersive velocity
vd = np.zeros((N,len(wi)))
for i in range(N):
    vd[i,:] = va[i]*np.cos(0.5*pi*ga[i])*(wi/w0)**ga[i]

In [ ]:
def nl(i,N,cmax=0.6):
    return np.sqrt(i/(N-1)*cmax)
xtk = np.linspace(5,50,6)
ytk = np.arange(800,2801,400)
# display the dispersive velocity
fig, ax = plt.subplots(1,1,figsize=(8,6))
for i in range(N):
    c = nl(i,N)
    if i == 0:
        ax.plot(wi/(2*pi),vd[i,:],color=(c,c,c),label='layer 1')
        continue
    if i == N-1:
        ax.plot(wi/(2*pi),vd[i,:],color=(c,c,c),label=f'layer {i+1}')
        continue
    if (i % 6 == 0) and (i!=30):
        ax.plot(wi/(2*pi),vd[i,:],color=(c,c,c),label=f'layer {i+1}')
    ax.plot(wi/(2*pi),vd[i,:],color=(c,c,c))
for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(2)
ax.legend(fontsize=15)
ax.set_xticks(xtk)
ax.set_xticklabels([f'{int(i)}' for i in xtk],fontsize=15)
ax.set_yticks(ytk)
ax.set_yticklabels([f'{i}' for i in ytk],fontsize=15)
ax.set_xlabel('Frequency (Hz)',fontsize=20)
_ = ax.set_ylabel('Velocity (m/s)',fontsize=20)


In [ ]:
fig.savefig('./outputs/AGUpaper/vdf_eps=0.01.png',dpi=600)

In [ ]:
# display returning residuals
y = testgb2.ABCtest(va, ga, disp=False)
nw = len(w)
Vmin = np.amin(y)
Vmax = np.amax(y)
cN = pcl.Normalize(vmin=Vmin, vmax=Vmax)
fig, ax = plt.subplots(1,1,figsize=(10,3.55))
ax.imshow(y,cmap='gray',origin='lower',\
          extent=[1,N,f0,f1],aspect=3/7*(N-1)*2*np.pi/(w[-1]-w[0]))
ax.set_xlabel('Absorbing layer No.',fontsize=20)
ax.set_ylabel('Frequency (Hz)',fontsize=20)
ax.set_xlim(1,N)
ax.set_ylim(5,50)
xtick = np.array(np.linspace(1,N,5),dtype=np.int16)
ytick = np.linspace(f0,f1,6)
ax.set_xticks(xtick)
ax.set_xticklabels([f'{i}' for i in xtick],fontsize=15)
ax.set_yticks(ytick)
ax.set_yticklabels([f'{i:.1f}' for i in ytick],fontsize=15)

cb = fig.colorbar(cm.ScalarMappable(norm=cN, cmap='gray'))
ctick = np.linspace(Vmin,Vmax,5)
cb.set_ticks(ctick)
cb.set_ticklabels([f'{i*1e2:.2f}' for i in ctick])
cb.ax.tick_params(labelsize=15)
pos = list(cb.ax.get_position().bounds)
pos[0] += 0.05
cb.ax.set_position(pos)
cb.set_label('$A_i(\omega)$',fontsize=15)

ax2 = ax.twinx()
color = 'tab:red'
ax2.set_ylabel('$\epsilon_i$', color=color, fontsize=20)
ax2.plot(x, epsi[1:], color=color, linewidth=2)
ax2.tick_params(axis='y', labelcolor=color)
ax2.set_ylim(0,0.01)
ytick = np.linspace(0,0.01,5)
ax2.set_yticks(ytick)
ax2.set_yticklabels([f'{i*1e2:.2f}' for i in ytick],fontsize=15)

ax2.text(32.3, 0.0105, 'x1e-2',fontsize=15,color=color)
ax2.text(38.3, 0.0105, 'x1e-2',fontsize=15)

for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(2)
#fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()

In [ ]:
BB = ax.get_position()

In [ ]:
BB.x0 = 0
BB.y0 = -0.3
BB.x1 = 10
BB.y1 = 3.55

In [ ]:
fig.savefig('./outputs/AGUpaper/Aepsi_eps=0.01.png',dpi=600,bbox_inches=BB)